In [2]:
import torch
import matplotlib.pyplot as plt
from utils import imageSize
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter


In [3]:

# Assuming you want to create a DataLoader for image datasets
train_transform = transforms.Compose([
    transforms.Resize((imageSize, imageSize)),
    transforms.ToTensor()
])
transform = transforms.Compose([
    transforms.Resize((imageSize, imageSize)),
    transforms.ToTensor()
])


In [ ]:
trainDataset = datasets.ImageFolder('data/train', transform=train_transform)
valDataset = datasets.ImageFolder('data/validation', transform=transform)
testDataset = datasets.ImageFolder('data/test', transform=transform)
batch_size = 30
num_workers = torch.get_num_threads()
train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valDataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testDataset, batch_size=batch_size, shuffle=True)

In [ ]:
clases = trainDataset.classes
classes =[longClass.split('-')[0] for longClass in trainDataset.classes]

In [ ]:
images, labels = next(iter(val_loader))
print(images.shape, labels.shape)

torch.Size([30, 3, 300, 300]) torch.Size([30])


# Create model

In [ ]:
%load_ext autoreload
%autoreload 2
from model_zoo import Animal2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_classes = len(classes)
model = Animal2(3, num_classes)
num_epochs = 100
valSteps = len(val_loader)
writer = SummaryWriter(comment='Change optim')
lossF = nn.CrossEntropyLoss()
learningRate = 1e-3
momentum = 0.9
optimizer = optim.ADAM(model.parameters(),learning_rate = learningRate)
accMetric = Accuracy(task='multiclass',num_classes=num_classes)
writer.add_hparams({'lr':learningRate,'optim':'Adam','Batch Size':batch_size,'momentum':momentum,'image size':imageSize},{'Number of epochs':num_epochs})

In [9]:
trainsteps  = len(train_loader)

In [10]:
modelSizeInBits = sum(p.numel() for p in model.parameters() if p.requires_grad) * 32
modelSizeInMB = modelSizeInBits/8/1e6
print(f'Model size: {modelSizeInMB} MB')
batchSizeInMB = images.element_size() * images.nelement() / 1e6
print(f'Batch size: {batchSizeInMB} MB')
labelsSizeInMB = labels.element_size() * labels.nelement() / 1e6
print(f'Labels size: {labelsSizeInMB} MB')

Model size: 89.81148 MB
Batch size: 32.4 MB
Labels size: 0.00024 MB


In [11]:
model = model.to(device)
writer.add_graph(model, images.to(device))

In [12]:
accMetric.to(device)
model.fit(train_loader,val_loader,criterion=lossF,optimizer=optimizer,\
          device=device,writer=writer,accMetric=accMetric,epochs=num_epochs,\
            num_classes=num_classes)

Epoch: 0
Training Loss: 1.7314 Training Accuracy: 0.3104
Validation Loss: 1.660770058631897 Validation Accuracy: 0.375
Epoch: 1
Training Loss: 1.6101 Training Accuracy: 0.4326
Validation Loss: 1.5917750597000122 Validation Accuracy: 0.4416666626930237
Epoch: 2
Training Loss: 1.5605 Training Accuracy: 0.4933
Validation Loss: 1.5503742694854736 Validation Accuracy: 0.5
Epoch: 3
Training Loss: 1.5293 Training Accuracy: 0.5156
Validation Loss: 1.5709577798843384 Validation Accuracy: 0.4833333194255829
Epoch: 4
Training Loss: 1.4940 Training Accuracy: 0.5570
Validation Loss: 1.5475128889083862 Validation Accuracy: 0.5166666507720947
Epoch: 5
Training Loss: 1.4788 Training Accuracy: 0.5756
Validation Loss: 1.5314165353775024 Validation Accuracy: 0.550000011920929
Epoch: 6
Training Loss: 1.4628 Training Accuracy: 0.5904
Validation Loss: 1.5037435293197632 Validation Accuracy: 0.550000011920929
Epoch: 7
Training Loss: 1.4483 Training Accuracy: 0.6022
Validation Loss: 1.5220673084259033 Validat

In [13]:
fineTuner = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
model.fit(train_loader,val_loader,criterion=lossF,optimizer=fineTuner,\
          device=device,writer=writer,accMetric=accMetric,epochs=20,\
            num_classes=num_classes,init_epoch=num_epochs)

Epoch: 45
Training Loss: 1.2623 Training Accuracy: 0.7963
Validation Loss: 1.4011534452438354 Validation Accuracy: 0.6583333015441895
Epoch: 46
Training Loss: 1.2578 Training Accuracy: 0.7933
Validation Loss: 1.3896548748016357 Validation Accuracy: 0.6583333015441895
Epoch: 47
Training Loss: 1.2470 Training Accuracy: 0.8074
Validation Loss: 1.3960061073303223 Validation Accuracy: 0.6499999761581421
Epoch: 48
Training Loss: 1.2506 Training Accuracy: 0.8067
Validation Loss: 1.3995065689086914 Validation Accuracy: 0.6499999761581421
Epoch: 49
Training Loss: 1.2459 Training Accuracy: 0.8074
Validation Loss: 1.3972867727279663 Validation Accuracy: 0.6416666507720947
Epoch: 50
Training Loss: 1.2479 Training Accuracy: 0.8089
Validation Loss: 1.3743996620178223 Validation Accuracy: 0.675000011920929
Epoch: 51
Training Loss: 1.2449 Training Accuracy: 0.8089
Validation Loss: 1.3767026662826538 Validation Accuracy: 0.6583333611488342
Epoch: 52
Training Loss: 1.2382 Training Accuracy: 0.8200
Valid